In [1]:
#  Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
# Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
# Повторить п.2, но используя уже не медиану, а max
# (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
# Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
# Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [2]:
import pandas as pd
from gensim.corpora.dictionary import Dictionary
import re
import numpy as np
from nltk.corpus import stopwords
import pymorphy2
from gensim.models import LdaModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
from razdel import tokenize
from gensim.test.utils import datapath
import itertools

%matplotlib inline

In [3]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [4]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [5]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [6]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [7]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n|n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    # tokens = list(tokenize(text))
    # words = [_.text for _ in tokens]
    # words = [w for w in words if w not in stopword_ru]
    
    # return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [8]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-7-5fdf025618b7>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 15.8 s


In [9]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 2min 10s


А теперь в 3 строчки обучим нашу модель

In [10]:
news['title']

0        [заместитель, председатель, правительство, рф,...
1        [матч, финал, кубок, россия, футбол, приостано...
2        [форвард, авангард, томаш, заборский, прокомме...
3        [главный, тренер, кубань, юрий, красножанин, п...
4        [решение, попечительский, совет, владивостокск...
                               ...                        
26995    [учёный, токийский, университет, морской, наук...
26996    [глава, кафедра, отечественный, история, xx, в...
26997    [американский, учёный, уточнить, возраст, расп...
26998    [последний, год, тропический, углеродный, цикл...
26999    [жить, примерно, тыс, год, назад, территория, ...
Name: title, Length: 27000, dtype: object

In [11]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [12]:
# common_dictionary.save('com_dict')

Все просто - это словарь наших слов

Запускаем обучение

In [13]:
# LdaModel?

In [14]:
%%time
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 20.6 s


In [15]:
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [16]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(0, 0.44121447),
 (2, 0.12817653),
 (14, 0.25449815),
 (16, 0.034108065),
 (20, 0.04666424),
 (21, 0.05141371),
 (24, 0.027453009)]

In [17]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: это мочь очень всё свой наш весь
topic_1: ск флот макаров малое арабский аргумент киргизия
topic_2: ребёнок планета бизнесмен мальчик дождь кость канал
topic_3: россия президент российский газ год глава экономика
topic_4: журнал достигать рекомендовать определение производить анализ фаза
topic_5: год страна это украина который россия развитие
topic_6: военный смерть километр боевой противник огонь конфликт
topic_7: банк болезнь орган это долг лицо социальный
topic_8: исследование который это год наука время весь
topic_9: цена квартира лекарство высота перевод продажа билет
topic_10: год который рубль компания строительство млн эксперт
topic_11: который сша американский статья россия человек это
topic_12: ракета спрос рынок турецкий станция сектор сенатор
topic_13: год который сша это северный земля мочь
topic_14: год млн тыс составить стать первый самый
topic_15: экипаж запуск двигатель машина пища отель парка
topic_16: станция фестиваль мероприятие дональд программа гость про

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [18]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [19]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.0,0.000000,0.670725,0.0,0.000000,0.0,0.136825,0.0,...,0.051081,0.048225,0.00000,0.075945,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,4896,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.00000,0.059242,0.000000,0.739897,0.000000,0.000000,0.0,0.000000
2,4897,0.441196,0.0,0.128213,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.034110,0.00000,0.000000,0.000000,0.046697,0.051423,0.000000,0.0,0.027446
3,4898,0.147337,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.71636,0.000000,0.026961,0.000000,0.000000,0.000000,0.0,0.000000
4,4899,0.000000,0.0,0.000000,0.339611,0.0,0.405233,0.0,0.000000,0.0,...,0.096052,0.000000,0.00000,0.000000,0.067596,0.000000,0.000000,0.068616,0.0,0.000000


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [20]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [21]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [22]:
doc_dict[293622]

array([0.04092521, 0.        , 0.03826533, 0.07590085, 0.        ,
       0.09206125, 0.        , 0.        , 0.17640802, 0.05990782,
       0.        , 0.12634881, 0.        , 0.        , 0.20327406,
       0.        , 0.08855224, 0.        , 0.        , 0.        ,
       0.04620054, 0.        , 0.04390511, 0.        , 0.        ])

In [23]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list, agg='mean'):
    if agg not in ['mean', 'max', 'median']:
        raise AttributeError("Вводимое значение должно быть одним из следующих:"\
                            "'mean', 'max', 'median'")
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    if agg == 'mean':
        user_vector = np.mean(user_vector, 0)
    elif agg == 'max':
        user_vector = np.nanmax(user_vector, 0)
    elif agg == 'median':
        user_vector = np.quantile(user_vector, 0.5, 0)
    return user_vector

In [24]:
get_user_embedding(user_articles_list)

array([0.06592716, 0.        , 0.00999803, 0.16495157, 0.00855178,
       0.07785631, 0.04026632, 0.01132301, 0.0496855 , 0.        ,
       0.07208936, 0.13709436, 0.00320983, 0.01909068, 0.03504364,
       0.00530104, 0.        , 0.08084617, 0.14167368, 0.        ,
       0.        , 0.0236455 , 0.03723724, 0.        , 0.00300934])

Интересовался новостями с топиками topic_3, topic_14 (что-то про политику и государство)

In [25]:
users['articles'].iloc[33]

'[323329, 321961, 324743, 323186, 324632, 474690]'

In [26]:
" ".join(news[news['doc_id']==323186]['title'].iloc[0])

'глава российский мид сергей лавров опровергнуть появиться сми информация якобы готовиться обмен декларация россия сша сотрудничество сфера сообщать риа новость читать сообщение разговаривать автор сообщение знать откуда автор источник какихлибо основание подобный род репортаж знать откуда информация появиться сказать журналист итог встреча госсекретарь сша джон керри позиция свой изложить декларация напринимать достаточно рамка обсе рамка совет россия нато высокий уровень продекларировать всё обеспечивать неделимость безопасность никто обеспечивать свой безопасность счёт безопасность продолжить министр слово лавров москва считать система нато создавать проблема наш безопасность поэтому декларация недостаточно мочь договариваться совместный система россия предлагать ещё начинать год президент путин посещать сша нужно вести речь очередной декларация гарантия который проверять объективный военнотехнический критерий гарантия ненаправленность система против российский ядерный потенциал под

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [27]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.020923,0.009586,0.040199,0.068535,0.000000,0.020985,0.000000,0.034615,0.074034,...,0.000000,0.046656,0.055149,0.106677,0.007611,0.031395,0.015035,0.083561,0.007925,0.013380
1,u108690,0.067786,0.000000,0.012530,0.113624,0.003758,0.129828,0.003607,0.038245,0.034414,...,0.000000,0.000000,0.156984,0.129472,0.012775,0.045777,0.000000,0.035417,0.000000,0.009262
2,u108339,0.001798,0.003219,0.022220,0.071126,0.006264,0.108985,0.010278,0.025518,0.070430,...,0.003131,0.019904,0.095601,0.118139,0.000000,0.075564,0.019337,0.051590,0.000000,0.003668


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [28]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [29]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.020923,0.009586,0.040199,0.068535,0.000000,0.020985,0.000000,0.034615,0.074034,...,0.046656,0.055149,0.106677,0.007611,0.031395,0.015035,0.083561,0.007925,0.013380,0
1,u108690,0.067786,0.000000,0.012530,0.113624,0.003758,0.129828,0.003607,0.038245,0.034414,...,0.000000,0.156984,0.129472,0.012775,0.045777,0.000000,0.035417,0.000000,0.009262,1
2,u108339,0.001798,0.003219,0.022220,0.071126,0.006264,0.108985,0.010278,0.025518,0.070430,...,0.019904,0.095601,0.118139,0.000000,0.075564,0.019337,0.051590,0.000000,0.003668,1


In [33]:
agg_list = ['mean', 'median', 'max']

target = pd.read_csv("users_churn.csv")

for agg in agg_list:
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, agg), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
    
    X = pd.merge(user_embeddings, target, 'left')
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                        X['churn'], random_state=0)
    logreg = LogisticRegression() 
    logreg.fit(X_train, y_train)
    preds = logreg.predict_proba(X_test)[:, 1]
    
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    ix = np.argmax(fscore)
    print('Agg method used:', agg)
    print('Best Threshold=%f, Roc-AUC=%.3f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix],
                                                                                          roc_auc_score(y_test, preds),
                                                                                          fscore[ix],
                                                                                          precision[ix],
                                                                                          recall[ix]))
    print('-' * 50)
    
#     roc_auc, precision, recall, f_score

Agg method used: mean
Best Threshold=0.263818, Roc-AUC=0.960, F-Score=0.713, Precision=0.652, Recall=0.788
--------------------------------------------------
Agg method used: median
Best Threshold=0.273887, Roc-AUC=0.977, F-Score=0.792, Precision=0.733, Recall=0.861
--------------------------------------------------
Agg method used: max
Best Threshold=0.316323, Roc-AUC=0.971, F-Score=0.768, Precision=0.725, Recall=0.816
--------------------------------------------------
